In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../src")

In [6]:
from architectures import SimpleCNN, MLP, ClassificationHead

In [5]:
cnn = SimpleCNN(
    in_channels=3,
    out_channels=[16, 32, 64],
    kernels=[3, 3, 3],
    pool_kernels=[3, 1, 3]
)

In [ ]:
mlp = MLP(
    in_dim = 
)